In [103]:
import ee
import geemap

In [104]:
ee.Initialize()

In [105]:
geometry = ee.Geometry.Polygon([[
   [33.58, 5.54], [ 33.90,-0.94], [ 39.22,-4.72],[41.60,-1.61], [ 41.90,4.00], [36.18,5.77],[33.54,5.54]
]])
#    [33.58, 5.54], [ 33.90,-0.94], [ 39.22,-4.72],[41.60,-1.61], [ 41.90,4.00], [36.18,5.77],[33.54,5.54]


In [113]:
aoi = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Kenya')).geometry()

In [114]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10 #bitwise left shift
    cirrusBitMask = 1 << 11 
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image)

In [203]:
def getEVI(image):
    EVI = image.expression(
        '2.5 * ((NIR-RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR' : image.select('B8').divide(10000),
            'RED' : image.select('B4').divide(10000),
            'BLUE' : image.select('B2').divide(10000),
        }).rename("EVI")
    image = image.addBands(EVI)
    return(image)

In [204]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [117]:
sentinel_data = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2022-05-01','2022-10-30') \
    .map(evi_calc).map(addDate).median()
#     .map(maskS2clouds) 

In [118]:
map1 = geemap.Map()

In [122]:
# palette = [
#     'blue', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
#     '74A901', '66A000', '529400', '3E8601', '207401', '056201',
#     '004C00', '023B01', '012E01', '011D01', '011301']
palette=['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
pall = {"min":0, "max":1, 'palette':palette}
map1.centerObject(aoi, 6)
# map1.addLayer(sentinel_data.select('EVI'), pall, "EVI")
map1.addLayer(sentinel_data.clip(aoi).select('EVI'),pall,"yooh1")
map1.addLayerControl()
map1

Map(bottom=33133.0, center=[0.5578063517900957, 37.84559934447033], controls=(WidgetControl(options=['position…

In [146]:
sentinel_img = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2022-05-01','2022-10-30')
image = sentinel_img.select('B4', 'B3', 'B2')


In [147]:
map2 = geemap.Map()
map2.addLayer(image,{'min':100,'max':3000},"yoh")
map2

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# works

In [160]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10 #bitwise left shift
    cirrusBitMask = 1 << 11 
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

In [161]:
sentinel_img1 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2022-05-01','2022-10-30').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15)) \
    .map(maskS2clouds).median()


# image = sentinel_img1.select('B4', 'B3', 'B2')


In [162]:
map3 = geemap.Map()
map3.addLayer(sentinel_img1,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3},'fcc')
map3

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# works

In [170]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

In [175]:
sentinel_img2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2022-05-01','2022-10-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 34)).median()

image2 = sentinel_img2.select('B4', 'B3', 'B2')

In [177]:
map4 = geemap.Map()
map4.addLayer(image2.clip(aoi),{'min':100,'max':3000},"yoh")
map4

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# https://gis.stackexchange.com/questions/333883/removing-clouds-from-sentinel-2-surface-reflectance-in-google-earth-engine

In [ ]:
# https://gis.stackexchange.com/questions/426571/cloud-shadow-removal-for-sentinel-2

In [187]:
# # Generate 'clear_sky' Sentinel-2 images using SCL.
# def s2_clear_sky(image):
#     scl = image.select('SCL')
#     clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(11))
#     return image.updateMask(clear_sky_pixels)

# # Map 's2_clear_sky' function over the imageCollection.
# s2_data = sentinel_img2.map(s2_clear_sky);bbb

In [205]:
sentinel_img5 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(aoi) \
    .filterDate('2018-05-01','2018-10-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 34)) \
    .map(getEVI).map(addDate).median()

# .filterDate('2022-05-01','2022-10-30') \

In [206]:
palette = [
    'blue', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
    '74A901', '66A000', '529400', '3E8601', '207401', '056201',
    '004C00', '023B01', '012E01', '011D01', '011301']
# palette=['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
pall = {"min":0, "max":1, 'palette':palette}

map5 = geemap.Map()
map5.centerObject(aoi, 6)
# map1.addLayer(sentinel_data.select('EVI'), pall, "EVI")
map5.addLayer(sentinel_img5.clip(aoi).select('EVI'),pall,"yooh1")
map5

EEException: Image.select: Pattern 'EVI' did not match any bands.

In [35]:
export_collection = collection.select(['EVI']).toBands()

In [36]:
export_collection

In [58]:
task_config = {
    'scale': 30,  
#     'region': geometry
    }

task = ee.batch.Export.image(export_collection, 'evi_evi', task_config)

#         'folder': 'Kitui',
#         'maxPixels': 1e13,
#         'region': geometry,
#         'fileformat':'GeoTIFF',
#         'scale': 30
#     }
# )

In [59]:
task.start()

In [ ]:
collection = ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(geometry)
    .filterDate('2020-07-01','2020-07-29')